## 3. Подготовка PyTorch Geometric графа (Data object)

Этот блок:

1. Превращает node_df в:
    - матрицу признаков x
    - вектор таргетов y

2. Превращает edge_df в:
    - edge_index
    - edge_attr

3. Собирает всё в один torch_geometric.data.Data

In [6]:
# Подготовка графа для PyTorch Geometric

import numpy as np
import pandas as pd
import torch
from torch_geometric.data import Data
from sklearn.preprocessing import StandardScaler


def prepare_pyg_graph(node_df: pd.DataFrame, edge_df: pd.DataFrame, target_col='churn_rate'):
    """
    Превращает node_df и edge_df в Data объект PyTorch Geometric.
    Требования:
        node_df.index = node_id
        edge_df.index = (src_node, dst_node)

    Возвращает:
        graph: Data (x, edge_index, edge_attr, y)
        node_id_map: dict node_id -> index
    """

    # :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
    # 1. Мэппинг node_id → индекс
    # :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
    node_ids = list(node_df.index)
    node_id_map = {nid: i for i, nid in enumerate(node_ids)}

    # :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
    # 2. Разбор числовых фичей нод
    # :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

    # Вытащим embedding и временно уберём
    text_emb = node_df['text_embedding'].apply(lambda x: np.array(x, dtype=float))
    node_df_wo_emb = node_df.drop(columns=['text_embedding'])

    # Категориальные фичи кодируем числом
    node_cat_cols = []
    for col in node_df_wo_emb.columns:
        if node_df_wo_emb[col].dtype == 'object':
            node_cat_cols.append(col)
            node_df_wo_emb[col] = node_df_wo_emb[col].astype('category').cat.codes

    # Все числовые колонки
    numeric_cols = [c for c in node_df_wo_emb.columns if c != target_col]

    # Выделяем матрицу фичей нод
    X_numeric = node_df_wo_emb[numeric_cols].astype(float).values

    # Скейлинг
    scaler = StandardScaler()
    X_numeric = scaler.fit_transform(X_numeric)

    # Добавляем text embedding
    X_emb = np.vstack(text_emb.values)
    X = np.concatenate([X_numeric, X_emb], axis=1)

    # :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
    # 3. Таргет y
    # :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
    y = torch.tensor(node_df_wo_emb[target_col].astype(float).values, dtype=torch.float)

    # :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
    # 4. Графовые данные: edge_index
    # :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
    sources = [node_id_map[s] for s, d in edge_df.index]
    targets = [node_id_map[d] for s, d in edge_df.index]

    edge_index = torch.tensor([sources, targets], dtype=torch.long)

    # :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
    # 5. edge_attr (фичи связей)
    # :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
    edge_df_local = edge_df.copy()

    # категориальные в связях
    edge_cat_cols = []
    for col in edge_df_local.columns:
        if edge_df_local[col].dtype == 'object':
            edge_cat_cols.append(col)
            edge_df_local[col] = edge_df_local[col].astype('category').cat.codes

    edge_X = edge_df_local.astype(float).values
    edge_X = torch.tensor(edge_X, dtype=torch.float)

    # :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
    # 6. Финальный граф
    # :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
    graph = Data(
        x=torch.tensor(X, dtype=torch.float),
        edge_index=edge_index,
        edge_attr=edge_X,
        y=y
    )

    return graph, node_id_map, scaler, numeric_cols


In [10]:
edge_df = pd.read_csv("../../data/edge_df.csv")
node_df = pd.read_parquet("../../data/node_semantic_df.parquet")

graph, node_id_map, scaler, numeric_cols = prepare_pyg_graph(node_df, edge_df)

print(graph)

TypeError: cannot unpack non-iterable int object

### Что мы получили

Теперь у нас полностью готов объект Data, который можно отправлять в GraphSAGE/GCN/GAT:
- graph.x — признаки нод
- graph.edge_index — структура графа
- graph.edge_attr — признаки связей
- graph.y — churn_rate вашей ноды

In [11]:
def prepare_pyg_graph(node_df: pd.DataFrame, edge_df: pd.DataFrame, target_col='churn_rate',
                      src_col='src_node', dst_col='dst_node'):
    """
    Превращает node_df и edge_df в Data объект PyTorch Geometric.
    Требования:
        node_df.index = node_id
        edge_df имеет столбцы src_col и dst_col с идентификаторами узлов

    Возвращает:
        graph: Data (x, edge_index, edge_attr, y)
        node_id_map: dict node_id -> index
    """

    # :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
    # 1. Мэппинг node_id → индекс
    # :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
    node_ids = list(node_df.index)
    node_id_map = {nid: i for i, nid in enumerate(node_ids)}

    # :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
    # 2. Разбор числовых фичей нод
    # :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

    # Проверяем наличие text_embedding
    if 'text_embedding' in node_df.columns:
        text_emb = node_df['text_embedding'].apply(lambda x: np.array(x, dtype=float) if x is not None else np.zeros(128))
        node_df_wo_emb = node_df.drop(columns=['text_embedding'])
    else:
        # Если embedding нет, создаём нулевой вектор (нужно указать размерность)
        embedding_dim = 128  # Или любая другая размерность
        text_emb = pd.Series([np.zeros(embedding_dim)] * len(node_df), index=node_df.index)
        node_df_wo_emb = node_df.copy()

    # Категориальные фичи кодируем числом
    node_cat_cols = []
    for col in node_df_wo_emb.columns:
        if col == target_col:
            continue
        if node_df_wo_emb[col].dtype == 'object':
            node_cat_cols.append(col)
            node_df_wo_emb[col] = node_df_wo_emb[col].astype('category').cat.codes

    # Все числовые колонки
    numeric_cols = [c for c in node_df_wo_emb.columns if c != target_col]

    # Выделяем матрицу фичей нод
    X_numeric = node_df_wo_emb[numeric_cols].astype(float).values

    # Скейлинг
    scaler = StandardScaler()
    X_numeric = scaler.fit_transform(X_numeric)

    # Добавляем text embedding
    X_emb = np.vstack(text_emb.values)
    X = np.concatenate([X_numeric, X_emb], axis=1)

    # :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
    # 3. Таргет y
    # :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
    y = torch.tensor(node_df_wo_emb[target_col].astype(float).values, dtype=torch.float)

    # :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
    # 4. Графовые данные: edge_index
    # :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
    # Проверяем наличие нужных столбцов
    if src_col not in edge_df.columns or dst_col not in edge_df.columns:
        raise ValueError(f"edge_df должен содержать столбцы {src_col} и {dst_col}")
    
    sources = [node_id_map[s] for s in edge_df[src_col]]
    targets = [node_id_map[d] for d in edge_df[dst_col]]

    edge_index = torch.tensor([sources, targets], dtype=torch.long)

    # :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
    # 5. edge_attr (фичи связей)
    # :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
    # Убираем ID столбцы из фичей связей
    edge_attr_cols = [col for col in edge_df.columns if col not in [src_col, dst_col]]
    
    if len(edge_attr_cols) > 0:
        edge_df_local = edge_df[edge_attr_cols].copy()
        
        # категориальные в связях
        edge_cat_cols = []
        for col in edge_df_local.columns:
            if edge_df_local[col].dtype == 'object':
                edge_cat_cols.append(col)
                edge_df_local[col] = edge_df_local[col].astype('category').cat.codes
        
        edge_X = edge_df_local.astype(float).values
        edge_X = torch.tensor(edge_X, dtype=torch.float)
    else:
        edge_X = None

    # :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
    # 6. Финальный граф
    # :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
    graph = Data(
        x=torch.tensor(X, dtype=torch.float),
        edge_index=edge_index,
        edge_attr=edge_X,
        y=y
    )

    return graph, node_id_map, scaler, numeric_cols


# Использование
edge_df = pd.read_csv("../../data/edge_df.csv")
node_df = pd.read_parquet("../../data/node_semantic_df.parquet")

# Укажите правильные имена столбцов с узлами
graph, node_id_map, scaler, numeric_cols = prepare_pyg_graph(
    node_df, 
    edge_df,
    src_col='src_node',  # замените на реальное имя столбца
    dst_col='dst_node'   # замените на реальное имя столбца
)

print(graph)

KeyError: '0072f89b60d46ef6f2094949d8831f13'